# Chapter 9: 2D axissymmetric tri element
Daryl Logan: A first course in the finite element method, 6th edition, 2020, Chapter 9.

Note: This is not an isoparametric formulation and the way the stiffenss matrix is calculated at the end is also not the best.

In [1]:
import numpy as np
import sympy as sp
sp.init_printing()

### The 2D axissymmetric tri element

This is a triangular element with nodes: i, j, m.

Coordinates are: r, z for radial and vertical directions, respectively.

Each node hat 2 degrees of freedom (DOFs):
- u: radial displacement
- w: vertical displacement

The DOFs are ordered as follows: u, w.

### The physical properties of the element

In [2]:
E = 200e9  # Young's modulus in Pa
nu = 0.3  # Poisson's ratio

# Define symbolic physical coordinates
r1, z1, r2, z2, r3, z3 = sp.symbols('r1 z1 r2 z2 r3 z3')

# Example coordinates for the nodes
# physical_coords = {'P1': (0, 0),
#                    'P2': (1, 0),
#                    'P3': (0, 1)}  # example coordinates for the nodes
physical_coords = {'P1': (3, 0),
                   'P2': (4, 0),
                   'P3': (3, 1)}  # example coordinates for the nodes
# physical_coords = {'P1': (0, 0),
#                    'P2': (3, 0),
#                    'P3': (0, 2)}  # example coordinates for the nodes

physical_coords_map = {'r1': physical_coords['P1'][0], 'z1': physical_coords['P1'][1],
                                 'r2': physical_coords['P2'][0], 'z2': physical_coords['P2'][1],
                                  'r3': physical_coords['P3'][0], 'z3': physical_coords['P3'][1]}

R, Z = sp.symbols('R Z')  # radial and vertical coordinates

# Define the physical nodes as a matrix so it can be used a symbolic evaluation
physical_nodes = sp.Matrix([[r1, z1], 
                            [r2, z2], 
                            [r3, z3]])

In [3]:
ND = 2  # number of degrees of freedom per node
NNODE = 3  # number of nodes

### Shape functions and derivatives

In [4]:
%matplotlib inline

def alpha():
    return sp.Matrix([
        r2 * z3 - r3 * z2,
        r3 * z1 - r1 * z3,
        r1 * z2 - r2 * z1,
    ])

def beta():
    return sp.Matrix([
        z2 - z3,
        z3 - z1,
        z1 - z2,
    ])

def gamma():
    return sp.Matrix([
        r3 - r2,
        r1 - r3,
        r2 - r1,
    ])


### Area of the element

In [5]:
# The double area of the triangle
two_A = sp.Matrix([r1, r2, r3]).dot(beta())
print('The value of two_A is:')
sp.pprint(two_A.subs(physical_coords_map))

# The single area of the triangle
Ae = sum(0.5 * sp.Matrix(alpha()))
print('The value of Ae is:')
sp.pprint(Ae.subs(physical_coords_map))

The value of two_A is:
1
The value of Ae is:
0.500000000000000


In [6]:

# # this is an alternative way of calculating the area of the triangle
# # using the determinant of the matrix formed by the coordinates of the nodes
# Ae = sum(0.5 * alpha())
# print('The value of Ae is:')
# sp.pprint(Ae.subs(physical_coords_map))

def shape_functions():
    _alpha = alpha()
    _beta = beta()
    _gamma = gamma()


    return sp.Matrix([
        (_alpha[0] + _beta[0] * R + _gamma[0] * Z) / two_A,
        (_alpha[1] + _beta[1] * R + _gamma[1] * Z) / two_A,
        (_alpha[2] + _beta[2] * R + _gamma[2] * Z) / two_A,
        ])

# Calculate the derivatives by s and t, symbolically
def shape_function_derivatives() -> np.array:
    """
    Shape function derivatives.
    [0] = dNi/ds, [1] = dNi/dt
    """
    sh = shape_functions()  # N1, N2, N3
    dN_dr = []
    dN_dz = []

    for i in range(len(sh)):
        dN_dr.append(sp.diff(sh[i], R))
        dN_dz.append(sp.diff(sh[i], Z))

    return dN_dr, dN_dz

# The symbolic shape functions
N_vec = shape_functions()

# 2. Substituting the symbolic coordinates with the physical coordinates
# This will give us the shape functions for the specific element defined by the physical coordinates
N_specific_element = N_vec.subs(physical_coords_map)

print("Symbolic shape function for this element:")
sp.pprint(N_specific_element)

# At node 1 N = [1, 0, 0]
print("\nValues at Node 1:")
sp.pprint(N_specific_element.subs({'R': physical_coords['P1'][0], 'Z': physical_coords['P1'][1]}).T)

# At node 2 N = [0, 1, 0]
print("\nValues at Node 2:")
sp.pprint(N_specific_element.subs({'R': physical_coords['P2'][0], 'Z': physical_coords['P2'][1]}).T)

# At node 3 N = [0, 0, 1]
print("\nValues at Node 3:")
sp.pprint(N_specific_element.subs({'R': physical_coords['P3'][0], 'Z': physical_coords['P3'][1]}).T)

print('\nShape function derivatives:')
dN_dr, dN_dz = shape_function_derivatives()
for i in range(len(dN_dr)):
    print(f'dN{i+1}/dr = {dN_dr[i]}')
    print(f'dN{i+1}/dz = {dN_dz[i]}')

# the shape function derivatives for the specific element
dN_dr_e = dN_dr[0].subs(physical_coords_map)
dN_dz_e = dN_dz[0].subs(physical_coords_map)
sp.pprint(dN_dr_e)
sp.pprint(dN_dz_e)

Symbolic shape function for this element:
⎡-R - Z + 4⎤
⎢          ⎥
⎢  R - 3   ⎥
⎢          ⎥
⎣    Z     ⎦

Values at Node 1:
[1  0  0]

Values at Node 2:
[0  1  0]

Values at Node 3:
[0  0  1]

Shape function derivatives:
dN1/dr = (z2 - z3)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
dN1/dz = (-r2 + r3)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
dN2/dr = (-z1 + z3)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
dN2/dz = (r1 - r3)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
dN3/dr = (z1 - z2)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
dN3/dz = (-r1 + r2)/(r1*(z2 - z3) + r2*(-z1 + z3) + r3*(z1 - z2))
-1
-1


### The shape function matrix

The shape function matrix $N$ is a matrix that contains the shape functions for each node. It has the structure:
$$
N = \begin{bmatrix}
N_i & 0 & N_j & 0 & N_m & 0 \\
0 & N_i & 0 & N_j & 0 & N_m\\
\end{bmatrix}
$$

In [7]:
def N() -> np.array:
    """
    shape function matrix N.
    It has the structure:
    [[Ni, 0, Nj, 0, Nm, 0],
     [0, Ni, 0, Nj, 0, Nm]]

    """
    _N = shape_functions()  # the shape functions

    # preparing the return matrix
    _ret = sp.Matrix(ND, ND * NNODE, lambda i, j: 0)  # ND rows, ND * NNODE columns

    # filling the matrix
    for i in range(NNODE):
        for j in range(ND):
            _ret[j, j + ND * i] = _N[i]

    return _ret

_N = N()
print("The shape function matrix N is {}:".format(_N.shape))
print()
print('The shape function matrix N for this element:')
_N_e = _N.subs(physical_coords_map)
sp.pprint(_N_e)

print('\nThe shape function matrix N for this element at node 1:')
midpoint = (physical_coords['P1'][0] + physical_coords['P2'][0] + physical_coords['P3'][0]) / 3, \
            (physical_coords['P1'][1] + physical_coords['P2'][1] + physical_coords['P3'][1]) / 3
_N_e_node1 = _N_e.subs({'R': midpoint[0], 'Z': midpoint[1]})
sp.pprint(_N_e_node1.applyfunc(lambda x: x.round(4)))


The shape function matrix N is (2, 6):

The shape function matrix N for this element:
⎡-R - Z + 4      0       R - 3    0    Z  0⎤
⎢                                          ⎥
⎣    0       -R - Z + 4    0    R - 3  0  Z⎦

The shape function matrix N for this element at node 1:
⎡0.3333    0     0.3333    0     0.3333    0   ⎤
⎢                                              ⎥
⎣  0     0.3333    0     0.3333    0     0.3333⎦


### $\begin{bmatrix}B\end{bmatrix}$ matrix
The full B matrix

In [8]:
def B_symbolic():

    _alpha = alpha()
    _beta = beta()
    _gamma = gamma()

    B = sp.zeros(4, ND*NNODE)
    for i in range(NNODE):
        B[0, 2*i] = _beta[i] / two_A
        B[1, 2*i] = (_alpha[i] / R + _beta[i] + _gamma[i] * Z / R) / two_A
        B[2, 2*i+1] = _gamma[i] / two_A
        B[3, 2*i] = _gamma[i] / two_A
        B[3, 2*i+1] = _beta[i] / two_A

        # print this block only
        # print(f'B block for node {i+1}:')
        # sp.pprint(B[:, 2*i:2*i+2].subs(physical_coords_map))

    return B

# The symbolic B matrix
B = B_symbolic()

# the B matrix for this element
B_e = B.subs(physical_coords_map)
# the B matrix for this element at the midpoint
print('the B matrix for this element is')
sp.pprint(B_e)

print('the B matrix for this element at the midpoint')
sp.pprint(B_e.subs({'R': midpoint[0], 'Z': midpoint[1]}))

# the B matrix for this element at node 1. Note, the node 1 is at the origin (0, 0) so we get multiple nan values due to division by zero
print('the B matrix for this element at node 1. Note, the node 1 is at the origin (0, 0) so we get multiple nan values due to division by zero')
sp.pprint(B_e.subs({'R': physical_coords['P1'][0], 'Z': physical_coords['P1'][1]}))


the B matrix for this element is
⎡    -1      0     1    0  0  0⎤
⎢                              ⎥
⎢     Z   4          3     Z   ⎥
⎢-1 - ─ + ─  0   1 - ─  0  ─  0⎥
⎢     R   R          R     R   ⎥
⎢                              ⎥
⎢    0       -1    0    0  0  1⎥
⎢                              ⎥
⎣    -1      -1    0    1  1  0⎦
the B matrix for this element at the midpoint
⎡-1   0    1   0   0   0⎤
⎢                       ⎥
⎢0.1  0   0.1  0  0.1  0⎥
⎢                       ⎥
⎢ 0   -1   0   0   0   1⎥
⎢                       ⎥
⎣-1   -1   0   1   1   0⎦
the B matrix for this element at node 1. Note, the node 1 is at the origin (0, 0) so we get multiple nan values due to division by zero
⎡-1   0   1  0  0  0⎤
⎢                   ⎥
⎢1/3  0   0  0  0  0⎥
⎢                   ⎥
⎢ 0   -1  0  0  0  1⎥
⎢                   ⎥
⎣-1   -1  0  1  1  0⎦


### The constitutive matrix $\begin{bmatrix}D\end{bmatrix}$

In [9]:
def D_matrix():
    """
    The constitutive matrix for the axissymmetric tri element.
    """
    # Plane stress condition
    D = sp.zeros(4, 4)
    D[0, 0] = 1 - nu
    D[0, 1] = nu
    D[0, 2] = nu

    D[1, 0] = nu
    D[1, 1] = 1 - nu
    D[1, 2] = nu

    D[2, 0] = nu
    D[2, 1] = nu
    D[2, 2] = 1 - nu

    D[3, 3] = (1 - 2 * nu) / 2

    D *= E / ((1 - 2 * nu) * (1 + nu))

    return D

D = D_matrix()
sp.pprint(D.applyfunc(lambda x: round(x/1e9, 2)))
print(D.T == D)

⎡269.23  115.38  115.38    0  ⎤
⎢                             ⎥
⎢115.38  269.23  115.38    0  ⎥
⎢                             ⎥
⎢115.38  115.38  269.23    0  ⎥
⎢                             ⎥
⎣  0       0       0     76.92⎦
True


### Quadrature points and weights

In [10]:
gi_data = {
    1: {
        'point': ((1/3, 1/3),),  # midpoint of the triangle
        'weight': (1,)
    },
    3: {
        'point': ((1/6, 1/6), (4/6, 1/6), (1/6, 4/6), ),
        'weight': (1/3, 1/3, 1/3)
    },
}

### The stiffness matrix $\begin{bmatrix}K\end{bmatrix}$

In [11]:
def K_matrix_1(ng=1):
    """ Gauss integration, 1 or 3 integration points. This is method 1 from the book."""
        # Get the symbolic B and constant D matrices
    B_sym = B_symbolic()
    D = D_matrix()

    # Substitute the physical node coordinates into B
    B_element = B_sym.subs(physical_coords_map)

    # The shape functions N(R,Z) map from natural to physical coordinates
    # Note: These are the isoparametric shape functions, used here only for mapping.
    N_map = sp.Matrix([1 - R - Z, R, Z])
    r_map = N_map.dot(sp.Matrix([r1, r2, r3])).subs(physical_coords_map)
    z_map = N_map.dot(sp.Matrix([z1, z2, z3])).subs(physical_coords_map)

    # Element area
    det_J = Ae.subs(physical_coords_map)

    K = sp.zeros(ND * NNODE, ND * NNODE)

    # Loop over the Gauss points and weights for the chosen scheme
    for p, w in zip(gi_data[ng]['point'], gi_data[ng]['weight']):
        R_gp, Z_gp = p[0], p[1]  # Gauss point in natural coordinates

        # Find the physical coordinates (r, z) for the current Gauss point
        r_physical_at_gp = r_map.subs({R: R_gp, Z: Z_gp})
        z_physical_at_gp = z_map.subs({R: R_gp, Z: Z_gp})

        B_evaluated = B_element.subs({'R': r_physical_at_gp, 'Z': z_physical_at_gp})

        K += B_evaluated.T * D * B_evaluated * 2 * sp.pi * r_physical_at_gp * det_J * w

    return K


import time
sta = time.time()
K_gauss_1pt = np.asarray(K_matrix_1(1).evalf().applyfunc(lambda x: round(x/1e12, 6)))
print('Time taken to calculate the stiffness matrix by 1 GP integration:', time.time() - sta)

import time
sta = time.time()
K_gauss_3pt = np.asarray(K_matrix_1(3).evalf().applyfunc(lambda x: round(x/1e12, 6)))
print('Time taken to calculate the stiffness matrix by 3 GP integration:', time.time() - sta)


print("Stiffness matrix from 1-point Gauss quadrature (K_gauss_1pt):")
print(K_gauss_1pt)

print("\nStiffness matrix from 3-point Gauss quadrature (K_gauss_3pt):")
print(K_gauss_3pt)

# Compare the integration results
print("\nComparison of the two stiffness matrices:")
import pprint
pprint.pprint(K_gauss_1pt / K_gauss_3pt)  # should be close to 1

Time taken to calculate the stiffness matrix by 1 GP integration: 0.06284475326538086
Time taken to calculate the stiffness matrix by 3 GP integration: 0.09726881980895996
Stiffness matrix from 1-point Gauss quadrature (K_gauss_1pt):
[[3.411447 1.893011 -2.791184 -0.805537 -0.898173 -1.087474]
 [1.893011 3.624915 -1.329135 -0.805537 -0.926367 -2.819378]
 [-2.791184 -1.329135 3.089233 0 0.149024 1.329135]
 [-0.805537 -0.805537 0 0.805537 0.805537 0]
 [-0.898173 -0.926367 0.149024 0.805537 0.833730 0.120830]
 [-1.087474 -2.819378 1.329135 0 0.120830 2.819378]]

Stiffness matrix from 3-point Gauss quadrature (K_gauss_3pt):
[[3.427433 1.893011 -2.798469 -0.805537 -0.904446 -1.087474]
 [1.893011 3.624915 -1.329135 -0.805537 -0.926367 -2.819378]
 [-2.798469 -1.329135 3.100160 0 0.141740 1.329135]
 [-0.805537 -0.805537 0 0.805537 0.805537 0]
 [-0.904446 -0.926367 0.141740 0.805537 0.849716 0.120830]
 [-1.087474 -2.819378 1.329135 0 0.120830 2.819378]]

Comparison of the two stiffness matrices

In [12]:

def K_matrix_2():
    """
    Calculates the stiffness matrix for the non-isoparametric axissymmetric element
    by performing a double integration over the element's physical area.
    """
    B_sym = B_symbolic()
    D_sym = D_matrix()
    integrand = B_sym.T * D_sym * B_sym * 2 * sp.pi * R
    integrand_element = integrand.subs(physical_coords_map)

    # Perform the double integration over the triangular area.
    # Note: In sympy, the integration variables are specified from inner to outer.
    K = sp.integrate(integrand_element, (Z, 0, 4 - R), (R, 3, 4))  # The order of integration is dz first, then dr. Outer integral: dr from r=3 to r=4 Inner integral: dz from z=0 to z=4-r

    return K

# Calculate the stiffness matrix
import time
sta = time.time()
K_integrated = K_matrix_2()
print('Time taken to calculate the stiffness matrix by double integration:', time.time() - sta)
print("The fully integrated stiffness matrix K is:")
sp.pprint(K_integrated.evalf().applyfunc(lambda x: round(x/1e12, 4)))

# comparing the results of the two methods
# K_e is the stiffness matrix calculated using method 3 from the book
# K_integrated is the stiffness matrix calculated by performing a double integration over the element's physical

K_integrated_np = np.asarray(K_integrated.evalf().applyfunc(lambda x: round(x/1e12, 6)))
# ratio = K_gauss_3pt / K_integrated_np
# for i in range(ratio.shape[0]):
#     for j in range(ratio.shape[1]):
#         print(f'Element ({i},{j}):', round(ratio[i, j], 6))


Time taken to calculate the stiffness matrix by double integration: 3.1757895946502686
The fully integrated stiffness matrix K is:
⎡3.4274    1.893   -2.7986  -0.8055  -0.9043  -1.0875⎤
⎢                                                    ⎥
⎢ 1.893   3.6249   -1.3291  -0.8055  -0.9264  -2.8194⎥
⎢                                                    ⎥
⎢-2.7986  -1.3291  3.1003      0     0.1416   1.3291 ⎥
⎢                                                    ⎥
⎢-0.8055  -0.8055     0     0.8055   0.8055      0   ⎥
⎢                                                    ⎥
⎢-0.9043  -0.9264  0.1416   0.8055   0.8497   0.1208 ⎥
⎢                                                    ⎥
⎣-1.0875  -2.8194  1.3291      0     0.1208   2.8194 ⎦


In [13]:

def K_matrix_3():
    """
    The stiffness matrix for the axissymmetric tri element.
    This is method 3 from the book, and the same as the 1 point Gauss integration method.
    It evaluates the B matrix at the midpoint of the element and then calculates the stiffness matrix.
    """
    # K = sp.zeros(ND * NNODE, ND * NNODE)  # the stiffness matrix

    mp = midpoint
    B_e = B_symbolic().subs(physical_coords_map)
    B = B_e.subs({'R': mp[0], 'Z': mp[1]})  # evaluate B at the midpoint

    D = D_matrix()  # the constitutive matrix

    _A = Ae.subs(physical_coords_map)  # element area

    K = (B.T * D * B) * 2 * np.pi * mp[0] * _A

    return K

# The stiffness matrix for this element
K_e = K_matrix_3()
K_e_formatted = K_e.applyfunc(lambda x: (x/1e12).round(4))
print("The stiffness matrix K_3 for this element:")
sp.pprint(K_e_formatted)

The stiffness matrix K_3 for this element:
⎡3.4114    1.893   -2.7912  -0.8055  -0.8982  -1.0875⎤
⎢                                                    ⎥
⎢ 1.893   3.6249   -1.3291  -0.8055  -0.9264  -2.8194⎥
⎢                                                    ⎥
⎢-2.7912  -1.3291  3.0892      0      0.149   1.3291 ⎥
⎢                                                    ⎥
⎢-0.8055  -0.8055     0     0.8055   0.8055      0   ⎥
⎢                                                    ⎥
⎢-0.8982  -0.9264   0.149   0.8055   0.8337   0.1208 ⎥
⎢                                                    ⎥
⎣-1.0875  -2.8194  1.3291      0     0.1208   2.8194 ⎦


In [14]:

def eigen(K):
    """
    Calculate the eigenvalues and eigenvectors of the stiffness matrix K.
    """
    # Convert the stiffness matrix to a numpy array
    K_np = np.array(K).astype(np.float64)

    # the eigenvalues
    eigenvalues, eigenvectors = np.linalg.eigh(K_np)
    eigenvalues = sorted([float(x) for x in eigenvalues])
    eigenvalues = [round(ev, 5) for ev in eigenvalues]
    print('The eigenvalues of the stiffness matrix K are: {}'.format(eigenvalues))

    # print([round(ev, 5) for ev in eigenvalues])
    # print(eigenvectors.shape)
    # sp.pprint(eigenvectors)

    # # the eigenvectors for the near-zero eigenvalues
    # nearzero_eigenvalues = [ev for ev in eigenvalues if abs(ev) < 1e-10]
    # # printing the eigenvectors for the near-zero eigenvalues
    # print('The eigenvectors for the near-zero eigenvalues are:')
    # for i, ev in enumerate(nearzero_eigenvalues):
    #     print(f'Eigenvalue {i + 1}: {ev:.6f}')
    #     # print('Eigenvector:')
    #     # print(eigenvectors[:, i])
    print()



print("Eigenvalues and eigenvectors for the stiffness matrix K_gauss_1pt:")
eigen(K_gauss_1pt)
print("Eigenvalues and eigenvectors for the stiffness matrix K_gauss_3pt:")
eigen(K_gauss_3pt)
print("Eigenvalues and eigenvectors for the stiffness matrix K_integrated:")
eigen(K_integrated_np)

print(K_gauss_1pt / K_gauss_3pt)
print()
print(K_gauss_1pt / K_integrated_np)
print()
print(K_gauss_3pt / K_integrated_np)




Eigenvalues and eigenvectors for the stiffness matrix K_gauss_1pt:
The eigenvalues of the stiffness matrix K are: [-0.0, 0.0, 0.0626, 2.13099, 3.22215, 9.1685]

Eigenvalues and eigenvectors for the stiffness matrix K_gauss_3pt:
The eigenvalues of the stiffness matrix K are: [0.0, 0.01079, 0.06332, 2.1416, 3.2325, 9.17893]

Eigenvalues and eigenvectors for the stiffness matrix K_integrated:
The eigenvalues of the stiffness matrix K are: [0.0, 0.01076, 0.06333, 2.14161, 3.23259, 9.17897]

[[0.9953359 1.000000 0.9973968 1.00000 0.993064 1.000000]
 [1.000000 1.000000 1.000000 1.00000 1.00000 1.000000]
 [0.9973968 1.000000 0.9964753 nan 1.05139 1.000000]
 [1.00000 1.00000 nan 1.00000 1.00000 nan]
 [0.993064 1.00000 1.05139 1.00000 0.981187 1.00000]
 [1.000000 1.000000 1.000000 nan 1.00000 1.000000]]

[[0.9953408 1.000000 0.9973587 1.00000 0.993240 1.000000]
 [1.000000 1.000000 1.000000 1.00000 1.00000 1.000000]
 [0.9973587 1.000000 0.9964236 nan 1.05219 1.000000]
 [1.00000 1.00000 nan 1.000